## Import all Libraries here

In [34]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_colwidth', 250)
pd.set_option('display.max_columns', 500)

#!pip install imblearn
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import RandomOverSampler 

## **Data Cleaning & Imputation**


In [35]:
ext_baseline3=pd.read_csv("ext_baseline3.csv")
ext_baseline3=ext_baseline3.drop(['Unnamed: 0','address(original)','size(original)','intercropCrop','cropVariety','machines(original)'],axis=1)
ext_baseline3=ext_baseline3.dropna(axis=0, how='all')
ext_baseline3=ext_baseline3.dropna(subset=["notPayupLoan"])
cleanup={"notPayupLoan":{"no":0,"yes":1}}
ext_baseline3=ext_baseline3.replace(cleanup)
ext_baseline3["numberOfLand"]=ext_baseline3["numberOfLand"].fillna(ext_baseline3["numberOfLand"].mean())
ext_baseline3["size"]=ext_baseline3["size"].fillna(ext_baseline3["size"].mean())
ext_baseline3["estimateMonthlyIncome"]=ext_baseline3["estimateMonthlyIncome"].fillna(ext_baseline3["estimateMonthlyIncome"].median())
ext_baseline3["applyLoanAmount"]=ext_baseline3["applyLoanAmount"].fillna(ext_baseline3["applyLoanAmount"].median())
ext_baseline3["yearsCultivating"]=ext_baseline3["yearsCultivating"].fillna(ext_baseline3["yearsCultivating"].median())
ext_baseline3["address"]=ext_baseline3["address"].fillna(ext_baseline3["address"].mode()[0])
ext_baseline3["intercropping"]=ext_baseline3["intercropping"].fillna(ext_baseline3["intercropping"].mode()[0])
ext_baseline3.address=ext_baseline3.address.replace({'Akasan taura ':'taura','Akasan taura':'taura'})
ext_baseline3.address=ext_baseline3.address.replace({'Gurjawa Kwalam ':'Gurjawa Kwalam'})  
ext_baseline3.crop=ext_baseline3.crop.replace({'Oil palm':'Oil Palm'})    
ext_baseline3.crop=ext_baseline3.crop.replace({'maize':'Maize'})    
ext_baseline3.machines=ext_baseline3.machines.replace({'n':'No','y':'Yes'})    
ext_baseline3.notPayupLoan=ext_baseline3.notPayupLoan.replace({'No':1,'Yes':0})    
ext_baseline3.notPayupLoan=ext_baseline3.notPayupLoan.astype('int')
ext_baseline3["notPayupLoan"].value_counts()

1    81
0     2
Name: notPayupLoan, dtype: int64

In [36]:
ext_baseline3.numberOfLand=ext_baseline3.numberOfLand.astype('int')
ext_baseline3.drop('size',axis=1,inplace=True)
ext_baseline3['income']=ext_baseline3.estimateMonthlyIncome.astype('int')
ext_baseline3.estimateMonthlyIncome[ext_baseline3.income<=50000]='below-50k'
ext_baseline3.estimateMonthlyIncome[(ext_baseline3.income>50000)&(ext_baseline3.income<=100000)]='50k-100k'
ext_baseline3.estimateMonthlyIncome[(ext_baseline3.income>100000)&(ext_baseline3.income<=250000)]='100k-250k'
ext_baseline3.estimateMonthlyIncome[(ext_baseline3.income>250000)&(ext_baseline3.income<=500000)]='250k-500k'
ext_baseline3.estimateMonthlyIncome[(ext_baseline3.income>500000)&(ext_baseline3.income<=1000000)]='500k-1m'
ext_baseline3.estimateMonthlyIncome[ext_baseline3.income>1000000]='1m-Above'
ext_baseline3.drop('income',axis=1,inplace=True)
ext_baseline3.estimateMonthlyIncome.value_counts()
ext_baseline3['amount']=ext_baseline3.applyLoanAmount.astype('int')
ext_baseline3.applyLoanAmount[ext_baseline3.amount<=50000]='below-50k'
ext_baseline3.applyLoanAmount[(ext_baseline3.amount>50000)&(ext_baseline3.amount<=100000)]='50k-100k'
ext_baseline3.applyLoanAmount[(ext_baseline3.amount>100000)&(ext_baseline3.amount<=250000)]='100k-250k'
ext_baseline3.applyLoanAmount[(ext_baseline3.amount>250000)&(ext_baseline3.amount<=500000)]='250k-500k'
ext_baseline3.applyLoanAmount[(ext_baseline3.amount>500000)&(ext_baseline3.amount<=1000000)]='500k-1m'
ext_baseline3.applyLoanAmount[ext_baseline3.amount>1000000]='1m-Above'
ext_baseline3.drop('amount',axis=1,inplace=True)
ext_baseline3['years']=ext_baseline3.yearsCultivating.astype('int')
ext_baseline3.yearsCultivating[ext_baseline3.years<=5]='below-5'
ext_baseline3.yearsCultivating[(ext_baseline3.years>5)&(ext_baseline3.years<=10)]='6-10'
ext_baseline3.yearsCultivating[(ext_baseline3.years>10)&(ext_baseline3.years<=20)]='11-20'
ext_baseline3.yearsCultivating[(ext_baseline3.years>20)&(ext_baseline3.years<=30)]='21-30'
ext_baseline3.yearsCultivating[(ext_baseline3.years>30)&(ext_baseline3.years<=40)]='31-40'
ext_baseline3.yearsCultivating[ext_baseline3.years>40]='41-Above'
ext_baseline3.drop('years',axis=1,inplace=True)

## Split Data before feature generation

In [37]:
from sklearn.model_selection import train_test_split
bl, bl_test = train_test_split(ext_baseline3, test_size=0.3, random_state=123)

In [38]:
bl_test.head()

,age,numberOfLand,address,owner_caretaker,crop,intercropping,machines,estimateMonthlyIncome,applyLoanAmount,yearsCultivating,notPayupLoan
103,45-54,1,taura,Owner,Sorghum,Yes,Yes,below-50k,100k-250k,11-20,1
109,25-34,3,Ercc,Owner,Soya bean,No,Yes,100k-250k,1m-Above,below-5,1
7,45-54,2,majia,Owner,sesame,Yes,Yes,50k-100k,100k-250k,11-20,1
114,25-34,5,Lagos,Owner,Cucumber,Yes,No,250k-500k,1m-Above,6-10,1
35,35-44,2,majia,Inherited,Rice,No,Yes,50k-100k,100k-250k,11-20,1


In [39]:
bl_test.columns

Index(['age', 'numberOfLand', 'address', 'owner_caretaker', 'crop',
       'intercropping', 'machines', 'estimateMonthlyIncome', 'applyLoanAmount',
       'yearsCultivating', 'notPayupLoan'],
      dtype='object')

## **Feature Engineering** 

In [40]:
def preprocess_yearsCultivating(df):
    cleanup={"yearsCultivating":{"below-5":1,"6-10":10,"11-20":20,"21-30":30,"31-40":40,"41-Above":50}}
    df=df.replace(cleanup)
    return(df)
def preprocess_estimateMonthlyIncome(df):
    cleanup={"estimateMonthlyIncome":{"below-50k":50,"50k-100k":100,"100k-250k":250,"250k-500k":500,"500k-1m":500,"1m-Above":1000}}
    df=df.replace(cleanup)
    return(df)
def preprocess_applyLoanAmount(df):
    cleanup={"applyLoanAmount":{"below-50k":50,"50k-100k":100,"100k-250k":250,"250k-500k":500,"500k-1m":500,"1m-Above":1000}}
    df=df.replace(cleanup)
    return(df)
def preprocess_intercropping(df):
    cleanup={"intercropping":{"No":0,"Yes":1}}
    df=df.replace(cleanup)
    return(df)
def preprocess_machines(df):
    cleanup={"machines":{"No":0,"Yes":1}}
    df=df.replace(cleanup)
    return(df)
def preprocess_owner_caretaker(df):
    cleanup={"owner_caretaker":{"Owner":0,"Inherited":1,"Caretaker":2}}
    df=df.replace(cleanup)
    return(df)
crop=['Cocoa', 'sesame', 'Sorghum', 'Rice', 'Millet', 'Maize', 'Tomato','Oil Palm', 'Cassava']
def preprocess_crop(df):
    for x in crop:
            df[x]=(df.crop==x).astype('int')
    df.drop('crop',axis=1,inplace=True)
    return df
age=['55-64', '35-44', '65-Above', '45-54', '25-34']
def preprocess_age(df):
    for x in age:
            df[x]=(df.age==x).astype('int')
    df.drop('age',axis=1,inplace=True)
    return df

address=['Bamikemo', 'taura', 'majia', 'Akasan gidan kuka',
       'Ibadan', 'wasai', 'garki', 'Gurjawa Kwalam', 'Kwatau Pampaida',
       'Ofatura']
def preprocess_address(df):
    for x in address:
            df[x]=(df.address==x).astype('int')
    df.drop('address',axis=1,inplace=True)
    return df

In [41]:
def feature_engineer(df):
    # Generate your features here!!!
    df.numberOfLand[df.numberOfLand=='6-Above']=6
    df.numberOfLand=df.numberOfLand.astype('int')
    df=preprocess_intercropping(preprocess_yearsCultivating(df))
    df=preprocess_machines(preprocess_estimateMonthlyIncome(df))
    df=preprocess_owner_caretaker(preprocess_applyLoanAmount(df))
    df=preprocess_crop(df)
    df=preprocess_age(df)
    df=preprocess_address(df)
    return df

In [42]:
# run feature engineering on train set
bl = feature_engineer(bl)
bl.head()

,numberOfLand,owner_caretaker,intercropping,machines,estimateMonthlyIncome,applyLoanAmount,yearsCultivating,notPayupLoan,Cocoa,sesame,Sorghum,Rice,Millet,Maize,Tomato,Oil Palm,Cassava,55-64,35-44,65-Above,45-54,25-34,Bamikemo,taura,majia,Akasan gidan kuka,Ibadan,wasai,garki,Gurjawa Kwalam,Kwatau Pampaida,Ofatura
98,2,0,1,0,100,50,20,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0
59,1,0,0,1,100,250,10,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0
97,2,0,1,1,100,250,20,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0
26,2,1,1,1,100,500,20,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0
17,2,0,1,1,100,250,30,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0


## Train & Save Model (you can name it with your notebook's name)

In [43]:
bl=bl.rename({'numberOfLand':'number_of_land','estimateMonthlyIncome':'estimate_monthly_income','yearsCultivating':'years_cultivating','applyLoanAmount':'apply_loan_amount'},axis=1)

In [44]:
X=bl.drop(["notPayupLoan"],axis=1)
y=bl["notPayupLoan"]

In [45]:
from sklearn.linear_model import LogisticRegression
classifier =  LogisticRegression()
classifier.fit(X, y)
### Create a Pickle file using serialization 
import pickle
pickle_out = open("modelExample.pkl","wb")
pickle.dump(classifier, pickle_out)
pickle_out.close()

## Load Test Set

In [46]:
# run feature engineering on test set
bl_test = feature_engineer(bl_test)

In [47]:
bl_test=bl_test.rename({'numberOfLand':'number_of_land','estimateMonthlyIncome':'estimate_monthly_income','yearsCultivating':'years_cultivating','applyLoanAmount':'apply_loan_amount'},axis=1)

In [48]:
X_test=bl_test.drop(["notPayupLoan"],axis=1)
y_test=bl_test["notPayupLoan"]

## Load Model

In [49]:
model = pickle.load(open('modelExample.pkl','rb'))

In [50]:
y_pred = model.predict_proba(X_test)[:,1]

In [51]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score,f1_score, roc_auc_score
#print(confusion_matrix(y_test,y_pred))

In [52]:
print(roc_auc_score(y_test, y_pred))

0.9166666666666666
